## UCL COMP0029 Individual Project for Year 3 BSc
### Robust Robotic Grasping Utilising Touch Sensing - Proposed Learning Framework Notebook
This notebook contains the essential code for training and testing a simplified learning framework for the proposed method - a simple multilayer perceptron to predict grasp outcomes.

### 1. Load packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import gc

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.decomposition import PCA

import seaborn as sns

Set device for `PyTorch` training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Empty PyTorch cache

In [ ]:
gc.collect()
torch.cuda.empty_cache()

### 2. Load datasets from saved .npy files

To collect data for this experiment, you can run the "Collect Sensory Data" button in the Pybullet simulation. This generates a predefined number of Gaussian grasps randomly generated from a base hand pose. Each individual grasp is considered as an individual experiment, and the data collected from this experiment is split into four, each stored in its own dataset.

For all object models used in this experiment, each object has 4 datasets which include:
- `depth_ds.npy` which stores the depth tactile data from the mounted DIGIT sensors
- `color_ds.npy` which stores the colored (RGB) version of the depth tactile data from the mounted DIGIT sensors
- `poses_ds.npy` which stores the randomly-generated 6d hand poses from the simulation
- `outcomes_ds.npy` which stores the outcomes of each random pose

In [ ]:
root = "../datasets/"

object_names = ["block1", "block2", "block3", "cylinder1", "cylinder2", "cylinder3"]

depth_data = np.empty((0, 2, 160, 120))
color_data = np.empty((0, 2, 160, 120, 3))
poses_data = np.empty((0, 6))
grasp_outcomes_data = np.empty((0,))

for object_name in object_names:
    # Construct the relative paths of each dataset and load them into the notebook
    depth_ds_file_path = root + object_name + "_ds/depth_ds.npy"
    depth_data1 = np.load(depth_ds_file_path)

    color_ds_file_path = root + object_name + "_ds/color_ds.npy"
    color_data1 = np.load(color_ds_file_path)

    poses_ds_file_path = root + object_name + "_ds/poses_ds.npy"
    poses_data1 = np.load(poses_ds_file_path)

    grasp_outcomes_ds_file_path = root + object_name + "_ds/grasp_outcomes.npy"
    grasp_outcomes_data1 = np.load(grasp_outcomes_ds_file_path)

    depth_data = np.append(depth_data, depth_data1, axis=0)
    color_data = np.append(color_data, color_data1, axis=0)
    poses_data = np.append(poses_data, poses_data1, axis=0)
    grasp_outcomes_data = np.append(grasp_outcomes_data, grasp_outcomes_data1, axis=0)

These datasets should all be in the form of $(N\times...)$ where $N$ is the number of examples:

In [ ]:
print(f"Shape of depth_data: {depth_data.shape}")
print(f"Shape of color_data: {color_data.shape}")
print(f"Shape of poses_data: {poses_data.shape}")
print(f"Shape of grasp_outcomes_data: {grasp_outcomes_data.shape}")

Additionally, we confirm the number of successful and unsuccessful grasps recorded. This helps us in the next section to determine how many examples we should include for each class in order to produce a balanced dataset.

In [ ]:
print(f"# of sucessesful grasps: {(grasp_outcomes_data == 1).sum()}")
print(f"# of unsuccessful grasps: {(grasp_outcomes_data == 0).sum()}")

### 3. Preprocessing

#### 3a. Sampling
We sample a pre-defined number of examples from each class label (successful and unsuccessful grasps) to reduce the computational cost.

In [ ]:
sample_size = 300

In [ ]:
# Function to sample n samples from the datasets
def sample_data(depth_data, color_data, poses_data, outcomes_data, no_of_examples):
    d, c, p, o = [], [], [], []
    for label in np.unique(outcomes_data):
        indices = np.where(outcomes_data == label)[0]

        for i in range(no_of_examples):
            d.append(depth_data[indices[i]])
            c.append(color_data[indices[i]])
            p.append(poses_data[indices[i]])
            o.append(outcomes_data[indices[i]])
    
    depth_data = torch.from_numpy(np.array(d))
    color_data = torch.from_numpy(np.array(c))
    poses_data = torch.from_numpy(np.array(p))
    outcomes_data = torch.from_numpy(np.array(o))
    return depth_data, color_data, poses_data, outcomes_data

We sample `sample_size` samples:

In [ ]:
depth_data, color_data, poses_data, grasp_outcomes_data = sample_data(depth_data, color_data, poses_data, grasp_outcomes_data, no_of_examples=sample_size)

In [ ]:
print(f"Shape of depth_data: {depth_data.shape}")
print(f"Shape of color_data: {color_data.shape}")
print(f"Shape of poses_data: {poses_data.shape}")
print(f"Shape of grasp_outcomes_data: {grasp_outcomes_data.shape}")
print(f"# of sucessesful grasps: {(grasp_outcomes_data == 1).sum()}")
print(f"# of unsuccessful grasps: {(grasp_outcomes_data == 0).sum()}")

#### 3b. Preprocessing

In [ ]:
def normalize(tensor):
    # Normalize & standardize each column
    mean = torch.mean(tensor, axis=0)
    std = torch.std(tensor, axis=0)
    tensor = (tensor - mean) / std
    tensor[torch.isnan(tensor)] = 0
    tensor[torch.isinf(tensor)] = 0
    return tensor


def downsample_2d_tensor(tensor, factor=2):
    return torch.nn.functional.avg_pool2d(tensor.unsqueeze(0), kernel_size=factor, stride=factor, padding=0).squeeze(0)


def downsample_3d_tensor(tensor, factor=2):
    pooling_kernel = 2
    pooling_layer = torch.nn.AvgPool2d(kernel_size=pooling_kernel, stride=pooling_kernel)
    downsampled_tensor = pooling_layer(tensor.permute(0, 3, 1, 2))
    downsampled_tensor = downsampled_tensor.permute(0, 2, 3, 1)
    
    return downsampled_tensor

#### 3c. Preparing datasets for training

We will use a tactile+visual data representation with CNNs as the input for our MLP approach. We concatenate the depth and color to get our tactile dataset, then concatenate the tactile and visual datasets together.

In [ ]:
depth_data = torch.cat([depth_data[:, 0, :, :], depth_data[:, 1, :, :]], dim=2)
color_data = torch.cat([color_data[:, 0, :, :, :], color_data[:, 1, :, :, :]], dim=2)
print(f"Shape of depth_data: {depth_data.shape}")
print(f"Shape of color_data: {color_data.shape}")

depth_ds = normalize(depth_data)
color_ds = normalize(color_data)
visual_ds = torch.from_numpy(np.nan_to_num(normalize(poses_data)))
visual_ds

In [ ]:
# tactile_ds = torch.cat([depth_ds, color_ds], dim=1)
# complete_ds = torch.cat([tactile_ds, visual_ds], dim=1)
tactile_ds = torch.cat([depth_ds.unsqueeze(-1), color_ds], dim=-1)
tactile_ds = torch.nan_to_num(tactile_ds)
complete_ds = torch.cat([tactile_ds.reshape(tactile_ds.shape[0], -1), visual_ds], dim=1)
complete_ds = torch.nan_to_num(complete_ds)
print(tactile_ds.shape, complete_ds.shape)

#### 3d. CNN dimensionality reduction

In [ ]:
# A simple convolutional neural network that extracts features from an input tensor
class FeatureExtractorCNN(nn.Module):
    def __init__(self):
        super(FeatureExtractorCNN, self).__init__()
        self.conv1 = nn.Conv2d(4, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=3)


    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        return x

In [ ]:
# Preprocess data using CNN feature extraction
cnn = FeatureExtractorCNN()
cnn_tactile = torch.cat([cnn(img.float().permute(2,0,1)).unsqueeze(0) for img in tactile_ds])
cnn_tactile = cnn_tactile.reshape(cnn_tactile.shape[0], -1)
cnn_tactile.shape

### 4. Testing different data representations

In our MLP model, there are two fully connected (dense) layers, each with an activation function (ReLU for the first layer and no activation for the second layer). The input size, hidden size, and output size are parameters that need to be specified when creating an instance of the MLP.

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x
    
    def train_mlp(self, epochs, X_train, y_train):
        self.losses = []
        for _ in range(epochs):
            inputs = torch.from_numpy(X_train).float()
            labels = y_train.float().view(-1, 1)

            # Zero the gradients
            self.optimizer.zero_grad()

            # Forward pass
            preds = self(inputs)
            loss = self.criterion(preds, labels)
            self.losses.append(loss.item())

            # Backward pass and optimization
            loss.backward()
            self.optimizer.step()
        
    def eval_mlp(self, X_test, y_test):
        # Evaluate the performance of the model on the testing set
        with torch.no_grad():
            inputs = torch.from_numpy(X_test).float()
            labels = y_test.float().view(-1, 1)

            # Forward pass
            final_preds = self(inputs)
            predicted = (final_preds > 0).float()
            
            # Confusion matrix
            cm = confusion_matrix(y_test, predicted)
            sns.heatmap(cm, linewidths=1, annot=True, fmt='g')

            # Accuracy
            accuracy = (predicted == labels).float().mean()
            print(f"Accuracy of MLP model: {accuracy*100:.2f}%")
    
    def plot_losses(self):
        plt.plot(self.losses)
        plt.xlabel('Epochs')
        plt.ylabel('Training Loss')
        plt.show


#### 4a. Tactile only + CNN -> MLP

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cnn_tactile.detach().numpy(), grasp_outcomes_data, test_size=0.2, random_state=0)

In [ ]:
mlp4a = MLP(input_size=512, hidden_size=64, output_size=1)
mlp4a.train_mlp(epochs=1000, X_train=X_train, y_train=y_train)
mlp4a.plot_losses()

In [ ]:
mlp4a.eval_mlp(X_test=X_test, y_test=y_test)

### 4b. Visual only + CNN -> MLP

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(visual_ds.detach().numpy(), grasp_outcomes_data, test_size=0.2, random_state=0)

In [ ]:
mlp4b = MLP(input_size=6, hidden_size=4, output_size=1)
mlp4b.train_mlp(epochs=1000, X_train=X_train, y_train=y_train)
mlp4b.plot_losses()

In [ ]:
mlp4b.eval_mlp(X_test=X_test, y_test=y_test)

### 5. Proposed method - All data -> MLP

We split our data into training and testing datasets for future sections.
We will only use the `complete_ds` for this approach. 

In [ ]:
# We simply combine the cnn-processed tactile data (from Section 5.3.1) with the visual data
cnn5_complete_ds = torch.cat([cnn_tactile.reshape(cnn_tactile.shape[0], -1), visual_ds], dim=1)
cnn5_complete_ds.shape

In [ ]:
X_cnn_complete_train, X_cnn_complete_test, y_cnn_complete_train, y_cnn_complete_test = train_test_split(cnn5_complete_ds.detach().numpy(), grasp_outcomes_data, test_size=0.2, random_state=0)

In [ ]:
mlp5 = MLP(input_size=518, hidden_size=64, output_size=1)
mlp5.train_mlp(epochs=1000, X_train=X_cnn_complete_train, y_train=y_cnn_complete_train)
mlp5.plot_losses()

In [ ]:
mlp5.eval_mlp(X_test=X_cnn_complete_test, y_test=y_cnn_complete_test)